In [3]:
import os
import io
import pandas as pd
from collections import namedtuple, OrderedDict

In [4]:
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util

In [14]:
import pickle 

fileObject = open('label_dict.p','rb') 
labels_dict = pickle.load(fileObject)

# switch order id:object order
class_labels = {}
i = 1
while i < len(labels_dict) + 1:
    class_labels[labels_dict[i]] = i
    i += 1
print(class_labels)

{'person': 1, 'bicycle': 2, 'car': 3, 'motorcycle': 4, 'airplane': 5, 'bus': 6, 'train': 7, 'truck': 8, 'boat': 9, 'traffic light': 10, 'fire hydrant': 11, 'stop sign': 12, 'parking meter': 13, 'bench': 14, 'bird': 15, 'cat': 16, 'dog': 17, 'horse': 18, 'sheep': 19, 'cow': 20, 'elephant': 21, 'bear': 22, 'zebra': 23, 'giraffe': 24, 'backpack': 25, 'umbrella': 26, 'handbag': 27, 'tie': 28, 'suitcase': 29, 'frisbee': 30, 'skis': 31, 'snowboard': 32, 'sports ball': 33, 'kite': 34, 'baseball bat': 35, 'baseball glove': 36, 'skateboard': 37, 'surfboard': 38, 'tennis racket': 39, 'bottle': 40, 'wine glass': 41, 'cup': 42, 'fork': 43, 'knife': 44, 'spoon': 45, 'bowl': 46, 'banana': 47, 'apple': 48, 'sandwich': 49, 'orange': 50, 'broccoli': 51, 'carrot': 52, 'hot dog': 53, 'pizza': 54, 'donut': 55, 'cake': 56, 'chair': 57, 'couch': 58, 'potted plant': 59, 'bed': 60, 'dining table': 61, 'toilet': 62, 'tv': 63, 'laptop': 64, 'mouse': 65, 'remote': 66, 'keyboard': 67, 'cell phone': 68, 'microwave

In [15]:
def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size
    
    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []
    
    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_labels[row['class']])
    
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [16]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [9]:
# load csv files
csv_trainval = 'train_fullcoco.csv'
csv_test = 'test_fullcoco.csv'
examples_trainval = pd.read_csv(csv_trainval)
grouped_trainval = split(examples_trainval, 'file_name')
examples_test = pd.read_csv(csv_test)
test_group = split(examples_test, 'file_name')

In [11]:
# shuffle and split into train/val and test
import random
random.seed(42)

random.shuffle(grouped_trainval)
val_percent = 0.1
val_group = grouped_trainval[:int(len(grouped_trainval)*val_percent)]
train_group = grouped_trainval[int(len(grouped_trainval)*val_percent):]

print('Total number of train val images: %s' % len(grouped_trainval))
print('Number of images in test group: %s' % len(test_group))
print('Number of images in train group: %s' % len(train_group))
print('Number of images in val group: %s' % len(val_group))

Total number of train val images: 117266
Number of images in test group: 4952
Number of images in train group: 105540
Number of images in val group: 11726


In [17]:
output_test_path = 'test_coco.record'
output_val_path = 'val_coco.record'
output_train_path = 'train_coco.record'
path = os.path.join(os.getcwd(), 'images/cooc2017')

writer_test = tf.python_io.TFRecordWriter(output_test_path)
for group in test_group:
    tf_example = create_tf_example(group, path)
    writer_test.write(tf_example.SerializeToString())
writer_test.close()
print('Successfully created the TFRecords: %s' % output_test_path)

writer_val = tf.python_io.TFRecordWriter(output_val_path)
for group in val_group:
    tf_example = create_tf_example(group, path)
    writer_val.write(tf_example.SerializeToString())
writer_val.close()
print('Successfully created the TFRecords: %s' % output_val_path)

writer_train = tf.python_io.TFRecordWriter(output_train_path)
for group in train_group:
    tf_example = create_tf_example(group, path)
    writer_train.write(tf_example.SerializeToString())
writer_train.close()
print('Successfully created the TFRecords: %s' % output_train_path)

Successfully created the TFRecords: test_coco.record
Successfully created the TFRecords: val_coco.record
Successfully created the TFRecords: train_coco.record
